In [1]:
import torch
import matplotlib.pylab as plt
import numpy as np
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import Compose,RandomHorizontalFlip,Normalize,ToTensor


In [3]:
# cifar_trainset = datasets.CIFAR10(root='./', train=True, download=True  )
# data = cifar_trainset.data / 255 # data is numpy array

# mean = data.mean(axis = (0,1,2)) 
# std = data.std(axis = (0,1,2))
# print(f"Mean : {mean}   STD: {std}")
# -> Mean : [0.49139968 0.48215841 0.44653091]   STD: [0.24703223 0.24348513 0.26158784]

Files already downloaded and verified
Mean : [0.49139968 0.48215841 0.44653091]   STD: [0.24703223 0.24348513 0.26158784]


In [2]:

mean = np.array([0.49139968, 0.48215841, 0.44653091])
std = np.array([0.24703223, 0.24348513, 0.26158784])
batch_size = 4
transforms = [ToTensor(), Normalize(std, mean)]

train_dataset = datasets.CIFAR10(
    root="./",
    train=True,
    transform=Compose([RandomHorizontalFlip(),*transforms]),
    download=True
    )

test_dataset = datasets.CIFAR10(
    root="./",
    train=False,
    transform=Compose(transforms),
    download=True
    )

train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

170499072it [00:35, 4795976.51it/s]                               


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [3]:
def im_convert(tensor):  
    image = tensor.cpu().clone().detach().numpy() # This process will happen in normal cpu.
    image = image.transpose(1, 2, 0)
    image = image * std + mean
    image = image.clip(0, 1)
    return image

In [4]:
dataiter = iter(train_loader)
images, labels = dataiter.next() 
fig = plt.figure(figsize=(25, 4)) 

# We plot images from our batch
for idx in np.arange(4):
  ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[]) 
  plt.imshow(im_convert(images[idx])) #converting to numpy array as plt needs it.
  ax.set_title(classes[labels[idx].item()])

: 

: 

In [ ]:
from torch.nn.functional import cross_entropy, relu
from torch.nn import MaxPool2d, Conv2d, Linear, Module, Dropout
from torch import flatten, no_grad, optim, save, load

In [7]:
class VGGNet(Module):
    def __init__(self, dropout=0.5):
        super(VGGNet, self).__init__()
        self.pool = MaxPool2d(2, 2)
        self.dropout = Dropout(dropout)
        self.conv1 = Conv2d(3, 32, 3, padding=1)
        self.conv2 = Conv2d(32, 32, 3, padding=1)

        self.conv3 = Conv2d(32, 64, 3, padding=1)
        self.conv4 = Conv2d(64, 64, 3, padding=1)

        self.conv5 = Conv2d(64, 128, 3, padding=1)
        self.conv6 = Conv2d(128, 128, 3, padding=1)
        self.conv7 = Conv2d(128, 128, 3, padding=1)

        self.fc1 = Linear(4*4*128, 512)
        self.fc2 = Linear(512, 256)
        self.fc3 = Linear(256, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool(relu(x))

        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool(relu(x))

        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool(relu(x))

        x = flatten(x, 1)
        x = self.dropout(relu(self.fc1(x)))
        x = self.dropout(relu(self.fc2(x)))
        x = relu(self.fc3(x))
        return x


In [11]:
MODEL_NAME = "myVGG.model"
LR = 1e-3
momentum = 0.9
EPOCH = 30
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:

def train(train_loader, epochs=30, lr=1e-3, momentum=0.9, device='cpu', model_name='myVGG', pre_train=False):
    model = VGGNet().to(device)
    if pre_train:
        try:
            model.load_state_dict(load(model_name))
            model.eval()
        except RuntimeError:
            print("model not found!")
            return
        
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    for epoch in range(epochs):
        loss = 0
        if epoch == 6:  # 6 epoch loss increase so reduce lr
            optimizer = optim.SGD(model.parameters(),
                                  lr=lr/10, momentum=momentum)

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            batchloss = cross_entropy(outputs, labels)
            batchloss.backward()

            optimizer.step()
            loss += batchloss.item()
        print(f"epoch: {epoch} loss: {loss / 2000:.3f}")
    save(model.state_dict(), model_name)

In [ ]:

def test(test_loader,device='cpu', model_name='myVGG'):
    total = len(test_loader.dataset)
    correct = 0
    model = VGGNet().to(device)
    try:
        model.load_state_dict(load(model_name))
        model.eval()
    except RuntimeError:
        print("model not found!")
        return

    with no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            y = model(images)
            pred_labels = y.max(dim = 1)[1]
            correct += (pred_labels == labels).sum().item()
    print("correct: ", correct)
    print("total: ", total)
    print(f"accuaracy: {correct/float(total)*100}%",)

In [ ]:

def test_every_classed(test_loader, classes, device='cpu', model_name='myVGG'):
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    model = VGGNet().to(device)
    model.load_state_dict(load(model_name))
    model.eval()
    with no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            y = model(images)
            pred_labels = y.max(dim = 1)[1]
            for label, prediction in zip(labels, pred_labels):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [ ]:
train(train_loader,device=DEVICE)


In [ ]:
test(test_loader,device=DEVICE)


In [15]:
test_every_classed(test_loader,classes,device=DEVICE)

correct:  8196
total:  10000
accuaracy: 81.96%
